In [1]:
import pandas as pd


In [2]:
data = pd.read_csv("acs2017_census_tract_data.csv")

In [5]:
data.iloc[:, :20].head()

,TractId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,Pacific,VotingAgeCitizen,Income,IncomeErr,IncomePerCap,IncomePerCapErr,Poverty,ChildPoverty,Professional
0,1001020100,Alabama,Autauga County,1845,899,946,2.4,86.3,5.2,0.0,1.2,0.0,1407,67826.0,14560.0,33018.0,6294.0,10.7,20.8,38.5
1,1001020200,Alabama,Autauga County,2172,1167,1005,1.1,41.6,54.5,0.0,1.0,0.0,1652,41287.0,3819.0,18996.0,2453.0,22.4,35.8,30.5
2,1001020300,Alabama,Autauga County,3385,1533,1852,8.0,61.4,26.5,0.6,0.7,0.4,2480,46806.0,9496.0,21236.0,2562.0,14.7,21.1,27.9
3,1001020400,Alabama,Autauga County,4267,2001,2266,9.6,80.3,7.1,0.5,0.2,0.0,3257,55895.0,4369.0,28068.0,3190.0,2.3,1.7,29.0
4,1001020500,Alabama,Autauga County,9965,5054,4911,0.9,77.5,16.4,0.0,3.1,0.0,7229,68143.0,14424.0,36905.0,10706.0,12.2,17.9,48.8


In [6]:
data.iloc[:, 20:].head()

,Service,Office,Construction,Production,Drive,Carpool,Transit,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,15.6,22.8,10.8,12.4,94.2,3.3,0.0,0.5,0.0,2.1,24.5,881,74.2,21.2,4.5,0.0,4.6
1,24.9,22.9,6.3,15.4,90.5,9.1,0.0,0.0,0.5,0.0,22.2,852,75.9,15.0,9.0,0.0,3.4
2,19.4,33.3,9.9,9.6,88.3,8.4,0.0,1.0,0.8,1.5,23.1,1482,73.3,21.1,4.8,0.7,4.7
3,16.6,25.8,9.1,19.5,82.3,11.2,0.0,1.5,2.9,2.1,25.9,1849,75.8,19.7,4.5,0.0,6.1
4,13.8,20.5,3.5,13.4,86.9,11.2,0.0,0.8,0.3,0.7,21.0,4787,71.4,24.1,4.5,0.0,2.3


In [7]:
data['State'].value_counts()

California              8057
Texas                   5265
New York                4918
Florida                 4245
Pennsylvania            3218
Illinois                3123
Ohio                    2952
Michigan                2813
North Carolina          2195
New Jersey              2010
Georgia                 1969
Virginia                1907
Arizona                 1526
Indiana                 1511
Tennessee               1497
Massachusetts           1478
Washington              1458
Wisconsin               1409
Maryland                1406
Missouri                1393
Minnesota               1338
Colorado                1249
Alabama                 1181
Louisiana               1148
Kentucky                1115
South Carolina          1103
Oklahoma                1046
Puerto Rico              945
Oregon                   834
Connecticut              833
Iowa                     825
Kansas                   770
Nevada                   687
Arkansas                 686
Mississippi   

In [13]:
data.groupby('State')['County'].nunique()

State
Alabama                  67
Alaska                   29
Arizona                  15
Arkansas                 75
California               58
Colorado                 64
Connecticut               8
Delaware                  3
District of Columbia      1
Florida                  67
Georgia                 159
Hawaii                    5
Idaho                    44
Illinois                102
Indiana                  92
Iowa                     99
Kansas                  105
Kentucky                120
Louisiana                64
Maine                    16
Maryland                 24
Massachusetts            14
Michigan                 83
Minnesota                87
Mississippi              82
Missouri                115
Montana                  56
Nebraska                 93
Nevada                   17
New Hampshire            10
New Jersey               21
New Mexico               33
New York                 62
North Carolina          100
North Dakota             53
Ohio          

In [14]:
data.groupby('State').count()

,TractId,County,TotalPop,Men,Women,Hispanic,White,Black,Native,Asian,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
State,,,,,,,,,,,,,,,,,,,,,
Alabama,1181,1181,1181,1181,1181,1176,1176,1176,1176,1176,...,1175,1175,1175,1175,1181,1175,1175,1175,1175,1175
Alaska,167,167,167,167,167,166,166,166,166,166,...,166,166,166,166,167,166,166,166,166,166
Arizona,1526,1526,1526,1526,1526,1520,1520,1520,1520,1520,...,1515,1515,1515,1510,1526,1515,1515,1515,1515,1515
Arkansas,686,686,686,686,686,685,685,685,685,685,...,685,685,685,683,686,685,685,685,685,685
California,8057,8057,8057,8057,8057,8012,8012,8012,8012,8012,...,7992,7992,7992,7972,8057,7990,7990,7990,7990,7990
Colorado,1249,1249,1249,1249,1249,1242,1242,1242,1242,1242,...,1237,1237,1237,1236,1249,1237,1237,1237,1237,1237
Connecticut,833,833,833,833,833,828,828,828,828,828,...,827,827,827,825,833,827,827,827,827,827
Delaware,218,218,218,218,218,214,214,214,214,214,...,214,214,214,214,218,214,214,214,214,214
District of Columbia,179,179,179,179,179,179,179,179,179,179,...,179,179,179,178,179,179,179,179,179,179
